# Monolith To MicroServices Udacity Project

In [1]:
import json
from IPython.utils.text import SList

## Setup Secrets

Copy the [sample.set_env.sh](./sample.set_env.sh) into a new file `set_env.sh`.

Edit the secrets to the appropriate values.

## Setup the AWS Project resources

Set up:

* The RDS (database)
* The S3 bucket (file storage)

Use CloudFormation to make it easier to handle all the related resources together.

Here is the resources dependency graph:

![Resources Dependency Graph](./screenshots/resources-dependency-graph.png)

In [11]:
%%bash
source ./set_env.sh
aws cloudformation create-stack \
  --template-body file://mono-to-micro-template.json \
  --parameters ParameterKey=DBPassword,ParameterValue=$POSTGRES_PASSWORD \
    ParameterKey=DBUsername,ParameterValue=$POSTGRES_USERNAME \
  --stack-name mono-to-micro-project

{
    "StackId": "arn:aws:cloudformation:us-east-1:549763406380:stack/mono-to-micro-project/da417c70-0fb7-11ed-883f-0ecf4ce8ae87"
}


In [8]:
project_stack: SList = !aws cloudformation describe-stacks --stack-name mono-to-micro-project

project_stack

['{',
 '    "Stacks": [',
 '        {',
 '            "StackId": "arn:aws:cloudformation:us-east-1:549763406380:stack/mono-to-micro-project/da417c70-0fb7-11ed-883f-0ecf4ce8ae87",',
 '            "StackName": "mono-to-micro-project",',
 '            "Parameters": [',
 '                {',
 '                    "ParameterKey": "DBPassword",',
 '                    "ParameterValue": "****"',
 '                },',
 '                {',
 '                    "ParameterKey": "DBUsername",',
 '                    "ParameterValue": "****"',
 '                }',
 '            ],',
 '            "CreationTime": "2022-07-30T03:29:43.401000+00:00",',
 '            "RollbackConfiguration": {},',
 '            "StackStatus": "CREATE_COMPLETE",',
 '            "DisableRollback": false,',
 '            "NotificationARNs": [],',
 '            "Outputs": [',
 '                {',
 '                    "OutputKey": "BucketName",',
 '                    "OutputValue": "mono-to-micro-project-s3bucket-sf6

In [9]:
project_stack: dict = json.loads(project_stack.nlstr)
project_stack

{'Stacks': [{'StackId': 'arn:aws:cloudformation:us-east-1:549763406380:stack/mono-to-micro-project/da417c70-0fb7-11ed-883f-0ecf4ce8ae87',
   'StackName': 'mono-to-micro-project',
   'Parameters': [{'ParameterKey': 'DBPassword', 'ParameterValue': '****'},
    {'ParameterKey': 'DBUsername', 'ParameterValue': '****'}],
   'CreationTime': '2022-07-30T03:29:43.401000+00:00',
   'RollbackConfiguration': {},
   'StackStatus': 'CREATE_COMPLETE',
   'DisableRollback': False,
   'NotificationARNs': [],
   'Outputs': [{'OutputKey': 'BucketName',
     'OutputValue': 'mono-to-micro-project-s3bucket-sf67os5rjm6b',
     'Description': 'The S3 Bucket created.'},
    {'OutputKey': 'DBEndpoint',
     'OutputValue': 'mr1fm3a15zjrrvp.crxb2aefz7m1.us-east-1.rds.amazonaws.com',
     'Description': 'The database address'},
    {'OutputKey': 'DBPort',
     'OutputValue': '5432',
     'Description': 'The database port'}],
   'Tags': [],
   'EnableTerminationProtection': False,
   'DriftInformation': {'StackDri

In [10]:
bucket_name, db_host, db_port = None, None, None
for output in project_stack['Stacks'][0]['Outputs']:
  if output['OutputKey'] == 'BucketName':
    bucket_name = output['OutputValue']
  elif output['OutputKey'] == 'DBEndpoint':
    db_host = output['OutputValue']
  elif output['OutputKey'] == 'DBPort':
    db_port = output['OutputValue']
    
bucket_name, db_host, db_port

('mono-to-micro-project-s3bucket-sf67os5rjm6b',
 'mr1fm3a15zjrrvp.crxb2aefz7m1.us-east-1.rds.amazonaws.com',
 '5432')

## Containers

Build the containers and push them to Docker Hub through a GitHub Action or Travis.

![Successful Build](./screenshots/successful-build.png)

![Docker Hub Images](./screenshots/docker-hub-images.png)

## Create the Kubernetes Cluster

In [6]:
%%bash
eksctl create cluster \
  --name monoToMicroProject \
  --region=us-east-1 \
  --nodes-min=2 \
  --nodes-max=3

2022-07-30 06:32:37 [ℹ]  eksctl version 0.107.0
2022-07-30 06:32:37 [ℹ]  using region us-east-1
2022-07-30 06:32:39 [ℹ]  skipping us-east-1e from selection because it doesn't support the following instance type(s): m5.large
2022-07-30 06:32:39 [ℹ]  setting availability zones to [us-east-1f us-east-1a]
2022-07-30 06:32:39 [ℹ]  subnets for us-east-1f - public:192.168.0.0/19 private:192.168.64.0/19
2022-07-30 06:32:39 [ℹ]  subnets for us-east-1a - public:192.168.32.0/19 private:192.168.96.0/19
2022-07-30 06:32:39 [ℹ]  nodegroup "ng-6db8eac3" will use "" [AmazonLinux2/1.22]
2022-07-30 06:32:39 [ℹ]  using Kubernetes version 1.22
2022-07-30 06:32:39 [ℹ]  creating EKS cluster "monoToMicroProject" in "us-east-1" region with managed nodes
2022-07-30 06:32:39 [ℹ]  will create 2 separate CloudFormation stacks for cluster itself and the initial managed nodegroup
2022-07-30 06:32:39 [ℹ]  if you encounter any issues, check CloudFormation console or try 'eksctl utils describe-stacks --region=us-east-

In [7]:
%%bash
kubectl get nodes

NAME                             STATUS   ROLES    AGE   VERSION
ip-192-168-28-69.ec2.internal    Ready    <none>   82s   v1.22.9-eks-810597c
ip-192-168-41-216.ec2.internal   Ready    <none>   78s   v1.22.9-eks-810597c


## Kubernetes Configs and Secrets

In [12]:
%%bash
kubectl apply -f aws-secret.yml

secret/app-aws-secret created


In [14]:
%%bash
kubectl apply -f env-secret.yml

secret/app-secret created


In [13]:
%%bash
kubectl apply -f env-configmap.yml

configmap/app-config created


## Kubernetes Deployment And Services

In [20]:
%%bash
kubectl apply -f deployments.yml
kubectl apply -f services.yml

deployment.apps/backend-user created
deployment.apps/backend-feed created
deployment.apps/reverseproxy created
deployment.apps/frontend created
service/backend-user created
service/backend-feed created
service/reverseproxy created
service/frontend created


In [22]:
%%bash
kubectl get pods

NAME                           READY   STATUS    RESTARTS      AGE
backend-feed-98dcfd6f9-lmwfs   1/1     Running   0             70s
backend-feed-98dcfd6f9-xj29c   1/1     Running   0             70s
backend-user-7996d768b-dqxhr   1/1     Running   0             72s
frontend-5db874bddf-6trb5      1/1     Running   0             67s
reverseproxy-89cf96df6-7c7jr   1/1     Running   2 (66s ago)   68s


In [23]:
%%bash
kubectl describe services

Name:              backend-feed
Namespace:         default
Labels:            <none>
Annotations:       <none>
Selector:          app=backend-feed
Type:              ClusterIP
IP Family Policy:  SingleStack
IP Families:       IPv4
IP:                10.100.248.101
IPs:               10.100.248.101
Port:              <unset>  8080/TCP
TargetPort:        8080/TCP
Endpoints:         192.168.24.9:8080,192.168.33.126:8080
Session Affinity:  None
Events:            <none>


Name:              backend-user
Namespace:         default
Labels:            <none>
Annotations:       <none>
Selector:          app=backend-user
Type:              ClusterIP
IP Family Policy:  SingleStack
IP Families:       IPv4
IP:                10.100.42.118
IPs:               10.100.42.118
Port:              <unset>  8080/TCP
TargetPort:        8080/TCP
Endpoints:         192.168.34.133:8080
Session Affinity:  None
Events:            <none>


Name:              frontend
Namespace:         default
Labels:            

## Port Forwarding

In [24]:
%%bash
kubectl port-forward service/reverseproxy 8080:8080 &
kubectl port-forward service/frontend 8100:8100 &

Forwarding from 127.0.0.1:8100 -> 80
Forwarding from [::1]:8100 -> 80
Forwarding from 127.0.0.1:8080 -> 8080
Forwarding from [::1]:8080 -> 8080
Handling connection for 8100
Handling connection for 8080
Process is interrupted.


![Port Forwarding](./screenshots/port-forward.png)

## Expose External IP

In [25]:
%%bash
kubectl expose deployment frontend \
  --type=LoadBalancer \
  --name=publicfrontend
kubectl expose deployment reverseproxy \
  --type=LoadBalancer \
  --name=publicreverseproxy

service/publicfrontend exposed
service/publicreverseproxy exposed


In [26]:
%%bash
kubectl get services

NAME                 TYPE           CLUSTER-IP       EXTERNAL-IP                                                               PORT(S)          AGE
backend-feed         ClusterIP      10.100.248.101   <none>                                                                    8080/TCP         12m
backend-user         ClusterIP      10.100.42.118    <none>                                                                    8080/TCP         12m
frontend             ClusterIP      10.100.118.30    <none>                                                                    8100/TCP         11m
kubernetes           ClusterIP      10.100.0.1       <none>                                                                    443/TCP          36m
publicfrontend       LoadBalancer   10.100.144.230   a139e703429fe46d2ada67281d885a44-1684113403.us-east-1.elb.amazonaws.com   80:31120/TCP     16s
publicreverseproxy   LoadBalancer   10.100.245.21    a59bb9bc6089c4fc98aba3844413e93e-1738954430.us-east-1.elb.a

### Rebuild Frontend

Using the exposed IPs, we replace them in the frontend environment files, then rebuild the images.

In [27]:
%%bash
cd ./udagram-frontend
docker build . -t inventrohyder/udagram-frontend:v1 --platform=linux/amd64
docker push inventrohyder/udagram-frontend:v1

#1 [internal] load build definition from Dockerfile
#1 sha256:9230c9cfd40bf868f9962ea253f7c36c25ba09b521909083950cff4ea958521b
#1 transferring dockerfile: 37B 0.0s done
#1 DONE 0.1s

#2 [internal] load .dockerignore
#2 sha256:1cb8d35d5ea853a7c7d4b9be7dc5ee9d4408ac0903365941be9f40c6501abb5a
#2 transferring context: 34B done
#2 DONE 0.1s

#4 [internal] load metadata for docker.io/library/nginx:alpine
#4 sha256:b001d263a254f0e4960d52c837d5764774ef80ad3878c61304052afb6e0e9af2
#4 ...

#5 [auth] library/nginx:pull token for registry-1.docker.io
#5 sha256:30e4b7261d14bc754ee85f53b40a4879e03a5f3fd17da3cb8a59db7f742e0981
#5 DONE 0.0s

#6 [auth] beevelop/ionic:pull token for registry-1.docker.io
#6 sha256:c56636257da5b2836ff5cc204093bad6c5fdac7dbbcc6858f7fdf0a72eb1c423
#6 DONE 0.0s

#3 [internal] load metadata for docker.io/beevelop/ionic:latest
#3 sha256:d4fa3a78ea0cdaa66ae5f73c6b8f351740f3988c736a74f62d31395d623ec9c3
#3 DONE 4.8s

#4 [internal] load metadata for docker.io/library/nginx:alpine


The push refers to repository [docker.io/inventrohyder/udagram-frontend]
f95643413040: Preparing
8dee52393d63: Preparing
d6cde4e7f474: Preparing
16669892469c: Preparing
1adb146a1535: Preparing
7e3d183a1226: Preparing
ec34fcc1d526: Preparing
7e3d183a1226: Waiting
16669892469c: Layer already exists
d6cde4e7f474: Layer already exists
8dee52393d63: Layer already exists
1adb146a1535: Layer already exists
ec34fcc1d526: Layer already exists
7e3d183a1226: Layer already exists
f95643413040: Pushed
v1: digest: sha256:b73ad991becfab89a4e27b9e6e148b8b27666d6566f4ac3cefe8c480e390f582 size: 1779


Update the [env-configmap.yml](./env-configmap.yml) to have the exposed frontend url.
Then load it.

In [28]:
%%bash
kubectl apply -f env-configmap.yml

configmap/app-config configured


Update the Kubernetes image to match the new one.

In [29]:
%%bash
kubectl set image deployment frontend frontend=inventrohyder/udagram-frontend:v1

deployment.apps/frontend image updated


In [30]:
%%bash
kubectl get pods

NAME                           READY   STATUS    RESTARTS      AGE
backend-feed-98dcfd6f9-lmwfs   1/1     Running   0             25m
backend-feed-98dcfd6f9-xj29c   1/1     Running   0             25m
backend-user-7996d768b-dqxhr   1/1     Running   0             25m
frontend-7bc4c74d-8h8cq        1/1     Running   0             41s
reverseproxy-89cf96df6-7c7jr   1/1     Running   2 (25m ago)   25m


![Exposed Endpoint Website](./screenshots/expose-endpoint.png)

## Auto-scale

### Deploy Metrics Server

As described within the following resources, we need the metric server first before setting up
a _Horizontal Pod Autoscaler_.

* <https://docs.aws.amazon.com/eks/latest/userguide/metrics-server.html>
* <https://docs.aws.amazon.com/eks/latest/userguide/horizontal-pod-autoscaler.html>

In [34]:
%%bash
kubectl apply \
  -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml

serviceaccount/metrics-server created
clusterrole.rbac.authorization.k8s.io/system:aggregated-metrics-reader created
clusterrole.rbac.authorization.k8s.io/system:metrics-server created
rolebinding.rbac.authorization.k8s.io/metrics-server-auth-reader created
clusterrolebinding.rbac.authorization.k8s.io/metrics-server:system:auth-delegator created
clusterrolebinding.rbac.authorization.k8s.io/system:metrics-server created
service/metrics-server created
deployment.apps/metrics-server created
apiservice.apiregistration.k8s.io/v1beta1.metrics.k8s.io created


Verify that the metrics server is running.

In [39]:
%%bash
kubectl get deployment metrics-server -n kube-system

NAME             READY   UP-TO-DATE   AVAILABLE   AGE
metrics-server   1/1     1            1           68s


In [41]:
%%bash
kubectl autoscale deployment backend-user \
  --cpu-percent=70 \
  --min=1 \
  --max=5
kubectl autoscale deployment backend-feed \
  --cpu-percent=70 \
  --min=2 \
  --max=5

horizontalpodautoscaler.autoscaling/backend-user autoscaled
horizontalpodautoscaler.autoscaling/backend-feed autoscaled


In [42]:
%%bash
kubectl describe hpa

Name:                                                  backend-feed
Namespace:                                             default
Labels:                                                <none>
Annotations:                                           <none>
CreationTimestamp:                                     Sat, 30 Jul 2022 08:09:28 +0300
Reference:                                             Deployment/backend-feed
Metrics:                                               ( current / target )
  resource cpu on pods  (as a percentage of request):  0% (0) / 70%
Min replicas:                                          2
Max replicas:                                          5
Deployment pods:                                       2 current / 2 desired
Conditions:
  Type            Status  Reason            Message
  ----            ------  ------            -------
  AbleToScale     True    ReadyForNewScale  recommended size matches current size
  ScalingActive   True    ValidMetricFound  th

## Logs

In [43]:
%%bash
kubectl logs backend-feed-98dcfd6f9-lmwfs


> udagram-api-feed@2.0.0 prod /usr/src/app
> tsc && node ./www/server.js

Initialize database connection...
Executing (default): CREATE TABLE IF NOT EXISTS "FeedItem" ("id"   SERIAL , "caption" VARCHAR(255), "url" VARCHAR(255), "createdAt" TIMESTAMP WITH TIME ZONE, "updatedAt" TIMESTAMP WITH TIME ZONE, PRIMARY KEY ("id"));
Executing (default): SELECT i.relname AS name, ix.indisprimary AS primary, ix.indisunique AS unique, ix.indkey AS indkey, array_agg(a.attnum) as column_indexes, array_agg(a.attname) AS column_names, pg_get_indexdef(ix.indexrelid) AS definition FROM pg_class t, pg_class i, pg_index ix, pg_attribute a WHERE t.oid = ix.indrelid AND i.oid = ix.indexrelid AND a.attrelid = t.oid AND t.relkind = 'r' and t.relname = 'FeedItem' GROUP BY i.relname, ix.indexrelid, ix.indisprimary, ix.indisunique, ix.indkey ORDER BY i.relname;
{"level":"info","message":"server running http://localhost:8100"}
{"level":"info","message":"press CTRL+C to stop server"}
{"contentLength":"-","id":"a41

## Clean up used resources

### Empty S3 Bucket

In [44]:
%%bash -s "$bucket_name"
aws s3 rm s3://$1 \
  --recursive

delete: s3://mono-to-micro-project-s3bucket-sf67os5rjm6b/xander0.jpg
delete: s3://mono-to-micro-project-s3bucket-sf67os5rjm6b/xander1.jpg


### Delete the CloudFormation Stack

In [45]:
%%bash
aws cloudformation delete-stack \
  --stack-name mono-to-micro-project

### Delete the Kubernetes cluster

In [46]:
%%bash
eksctl delete cluster --name monoToMicroProject --region=us-east-1

2022-07-30 09:43:00 [ℹ]  deleting EKS cluster "monoToMicroProject"
2022-07-30 09:43:03 [ℹ]  will drain 0 unmanaged nodegroup(s) in cluster "monoToMicroProject"
2022-07-30 09:43:03 [ℹ]  starting parallel draining, max in-flight of 1
2022-07-30 09:43:05 [ℹ]  deleted 0 Fargate profile(s)
2022-07-30 09:43:09 [✔]  kubeconfig has been updated
2022-07-30 09:43:09 [ℹ]  cleaning up AWS load balancers created by Kubernetes objects of Kind Service or Ingress
2022-07-30 09:44:40 [ℹ]  
2 sequential tasks: { delete nodegroup "ng-6db8eac3", delete cluster control plane "monoToMicroProject" [async] 
}
2022-07-30 09:44:41 [ℹ]  will delete stack "eksctl-monoToMicroProject-nodegroup-ng-6db8eac3"
2022-07-30 09:44:41 [ℹ]  waiting for stack "eksctl-monoToMicroProject-nodegroup-ng-6db8eac3" to get deleted
2022-07-30 09:44:41 [ℹ]  waiting for CloudFormation stack "eksctl-monoToMicroProject-nodegroup-ng-6db8eac3"
2022-07-30 09:45:13 [ℹ]  waiting for CloudFormation stack "eksctl-monoToMicroProject-nodegroup-ng-